# CSE151B Final Project- SpaiNN

#### Authors: Álvaro Maza Montesinos & Juan Villanueva Romero

## Import libraries

In [1]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import ast
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
# Avoids a warning we were getting when running the code
import warnings
warnings.filterwarnings("ignore")

## Preprocessing

First, we load the training dataset and the metadata provided in class:

In [2]:
# These are all of the files you are given
data = pd.read_csv("train.csv")

In [3]:
csv_df = pd.read_csv('metaData_taxistandsID_name_GPSlocation.csv', sep=',')

We realized that one value of the metadata was wrongly writen, so we had to correct it before proceding with the preprocessing

In [4]:
d = csv_df['Latitude'][40].split('-')
csv_df.at[40, 'Latitude'] = float(d[0])
csv_df.at[40, 'Longitude'] = -float(d[1])
csv_df.loc[40:45]

,ID,Descricao,Latitude,Longitude
40,41,Nevogilde,41.163067,-8.675983
41,42,Nove de Abril,41.1727485684,-8.611784
42,43,Ouro,41.1486110328,-8.648886
43,44,Palácio,41.149617396,-8.625784
44,45,Passeio Alegre,41.1486486007,-8.672795
45,46,Pereira de Melo,41.1786451618,-8.648536


The following two functions were already given in the feature_engineering notebook.

In [5]:
def polyline_to_trip_duration(polyline):
  return max(polyline.count("[") - 2, 0) * 15

def parse_time(x):
  # We are using python's builtin datetime library
  # https://docs.python.org/3/library/datetime.html#datetime.date.fromtimestamp

  # Each x is essentially a 1 row, 1 column pandas Series
  dt = datetime.fromtimestamp(x["TIMESTAMP"])
  return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

Now, let's display our data and identify any feature engineering that we can perform

In [6]:
data

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."
...,...,...,...,...,...,...,...,...,...
1710665,1404171463620000698,C,NaN,NaN,20000698,1404171463,A,False,"[[-8.612469,41.14602],[-8.612487,41.145993],[-..."
1710666,1404171367620000670,C,NaN,NaN,20000670,1404171367,A,False,"[[-8.610138,41.140845],[-8.610174,41.140935],[..."
1710667,1388745716620000264,C,NaN,NaN,20000264,1388745716,A,False,[]
1710668,1404141826620000248,B,NaN,12.0,20000248,1404141826,A,False,"[[-8.630712,41.154885],[-8.63073,41.154813],[-..."


In [7]:
data.isna().sum()

TRIP_ID               0
CALL_TYPE             0
ORIGIN_CALL     1345900
ORIGIN_STAND     904091
TAXI_ID               0
TIMESTAMP             0
DAY_TYPE              0
MISSING_DATA          0
POLYLINE              0
dtype: int64

As the first step, we can see that there exists a column indicating missing data in each row. Since we have over 1.5 million observations, we have decided to remove observations with NAs:

In [8]:
data = data[data['MISSING_DATA'] == False]

Also, now that it is no longer useful we will remove the MISSING_DATA column, as well as the TRIP_ID since it is unique for every observation, and the ORIGIN_CALL since almost all the column is full of NAs.

In [9]:
data = data.drop('MISSING_DATA', axis=1)
data = data.drop('TRIP_ID', axis=1)
#data = data.drop('CALL_TYPE', axis=1)
data = data.drop('ORIGIN_CALL', axis=1)

Finally, we will only keep the three last digits of the taxi ID:

In [10]:
data['TAXI_ID']=data['TAXI_ID'].astype(str).str[-3:]

In [11]:
data.head()

,CALL_TYPE,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,POLYLINE
0,C,NaN,589,1372636858,A,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,B,7.0,596,1372637303,A,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,C,NaN,320,1372636951,A,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,C,NaN,520,1372636854,A,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,C,NaN,337,1372637091,A,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


We can now perform a merge to include the metadata within our training dataset:

In [12]:
data = pd.merge(data, csv_df, left_on='ORIGIN_STAND', right_on='ID', how='left')

In [13]:
data

,CALL_TYPE,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,POLYLINE,ID,Descricao,Latitude,Longitude
0,C,NaN,589,1372636858,A,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",NaN,NaN,NaN,NaN
1,B,7.0,596,1372637303,A,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",7.0,Av. Boavista,41.1599801853,-8.641984
2,C,NaN,320,1372636951,A,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",NaN,NaN,NaN,NaN
3,C,NaN,520,1372636854,A,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",NaN,NaN,NaN,NaN
4,C,NaN,337,1372637091,A,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1710655,C,NaN,698,1404171463,A,"[[-8.612469,41.14602],[-8.612487,41.145993],[-...",NaN,NaN,NaN,NaN
1710656,C,NaN,670,1404171367,A,"[[-8.610138,41.140845],[-8.610174,41.140935],[...",NaN,NaN,NaN,NaN
1710657,C,NaN,264,1388745716,A,[],NaN,NaN,NaN,NaN
1710658,B,12.0,248,1404141826,A,"[[-8.630712,41.154885],[-8.63073,41.154813],[-...",12.0,Bom Sucesso,41.1551197292,-8.630466


In [14]:
data.isna().sum()

CALL_TYPE            0
ORIGIN_STAND    904084
TAXI_ID              0
TIMESTAMP            0
DAY_TYPE             0
POLYLINE             0
ID              904084
Descricao       904084
Latitude        904084
Longitude       904084
dtype: int64

Once again, we drop some unnecesary columns:

In [15]:
# Drop unnecessary columns
data.drop(['Descricao'], axis=1, inplace=True)
data.drop(['ID'], axis=1, inplace=True)
data.drop(['ORIGIN_STAND'], axis=1, inplace=True)

# Display the merged DataFrame
data.head()

,CALL_TYPE,TAXI_ID,TIMESTAMP,DAY_TYPE,POLYLINE,Latitude,Longitude
0,C,589,1372636858,A,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",NaN,NaN
1,B,596,1372637303,A,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",41.1599801853,-8.641984
2,C,320,1372636951,A,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",NaN,NaN
3,C,520,1372636854,A,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",NaN,NaN
4,C,337,1372637091,A,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...",NaN,NaN


We transform the polyline into the trip length by uisng the function provided in class

In [16]:
data["LEN"] = data["POLYLINE"].apply(polyline_to_trip_duration)

In [17]:
data.describe()

,TIMESTAMP,Longitude,LEN
count,1.710660e+06,806576.000000,1.710660e+06
mean,1.388622e+09,-8.613889,7.164222e+02
std,9.180948e+06,0.020803,6.847290e+02
min,1.372637e+09,-8.689180,0.000000e+00
25%,1.380731e+09,-8.627759,4.050000e+02
50%,1.388493e+09,-8.611071,6.000000e+02
75%,1.396750e+09,-8.602539,8.700000e+02
max,1.404173e+09,-8.568195,5.820000e+04


By looking at the data, we have decided to remove __outliers__ by setting lower and upper thresholds

In [18]:
threshold = 900
minimum_len = 30
data = data[(data['LEN'] <= threshold) & (data['LEN'] >= minimum_len)]

We can now fill the NAs in the latitude and longitude by using the first value in the polyline. We considered using median inputation, but it did not provide very good results.

In [19]:
first_longitude = data['POLYLINE'].apply(lambda x: (x.split(',')[0])[2:])
data.loc[:,'Longitude'] = data['Longitude'].fillna(pd.Series(first_longitude))

In [20]:
first_latitude = data['POLYLINE'].apply(lambda x: (x.split(',')[1])[:-1])
data.loc[:,'Latitude'] = data['Latitude'].fillna(pd.Series(first_latitude))

With respect to the timestamp, it gets separated into different columns, but we will only keep the day of the week and the hour, since we think it is the only relevant information in this case.

In [21]:
data[["YR", "MON", "DAY", "HR", "WK"]] = data[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")

In [22]:
data = data.drop('TIMESTAMP', axis=1)
data = data.drop('POLYLINE', axis=1)
data = data.drop('YR', axis=1)
data = data.drop('MON', axis=1)
data = data.drop('DAY', axis=1)

Finally, we have decided to group the hours of the day into intervals, since this makes it easier to have one-hot encoding:

In [23]:
bins = [-1, 4, 8, 12, 16, 20, 24]
labels = [0, 1, 2, 3, 4, 5]

data['HOUR_GROUP'] = pd.cut(data['HR'], bins=bins, labels=labels, include_lowest=True)
data.drop(['HR'], axis=1, inplace=True)
data

,CALL_TYPE,TAXI_ID,DAY_TYPE,Latitude,Longitude,LEN,WK,HOUR_GROUP
0,C,589,A,41.141412,-8.618643,330,0,0
1,B,596,A,41.1599801853,-8.641984,270,0,0
3,C,520,A,41.151951,-8.574678,630,0,0
4,C,337,A,41.18049,-8.645994,420,0,0
5,C,231,A,41.140674,-8.615502,375,0,0
...,...,...,...,...,...,...,...,...
1710649,A,213,A,41.157099,-8.574732,795,5,1
1710651,B,332,A,41.1486275073,-8.585877,480,0,5
1710655,C,698,A,41.14602,-8.612469,465,0,5
1710656,C,670,A,41.140845,-8.610138,435,0,5


In [24]:
data.dtypes

CALL_TYPE       object
TAXI_ID         object
DAY_TYPE        object
Latitude        object
Longitude       object
LEN              int64
WK               int64
HOUR_GROUP    category
dtype: object

In [25]:
columns_to_convert = ['Latitude', 'Longitude']
data[columns_to_convert] = data[columns_to_convert].astype(float)
data['WK'] = data['WK'].astype('category')

In [26]:
data.dtypes

CALL_TYPE       object
TAXI_ID         object
DAY_TYPE        object
Latitude       float64
Longitude      float64
LEN              int64
WK            category
HOUR_GROUP    category
dtype: object

In [28]:
# Split the data into training and testing sets
X = data.drop('LEN', axis=1)
y = data['LEN']

columns_to_scale = ['Latitude', 'Longitude']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the selected columns using StandardScaler
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[columns_to_scale] = scaler.fit_transform(X_train_scaled[columns_to_scale])
X_test_scaled[columns_to_scale] = scaler.transform(X_test_scaled[columns_to_scale])

In [29]:
X_train_scaled = pd.get_dummies(X_train_scaled)
X_test_scaled = pd.get_dummies(X_test_scaled)

In [30]:
X_train_scaled

,Latitude,Longitude,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C,TAXI_ID_001,TAXI_ID_002,TAXI_ID_003,TAXI_ID_004,TAXI_ID_005,...,WK_3,WK_4,WK_5,WK_6,HOUR_GROUP_0,HOUR_GROUP_1,HOUR_GROUP_2,HOUR_GROUP_3,HOUR_GROUP_4,HOUR_GROUP_5
1026448,-0.614876,0.173266,0,1,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
1105591,-0.186826,0.456551,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
164503,0.058017,-0.211671,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1186306,-0.444315,-1.753912,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
656806,-0.100573,-0.535776,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142069,0.128181,-0.855104,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
331200,0.258340,-1.578430,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
169429,-0.020282,-0.168158,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
888732,-0.109308,0.147813,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
